<img src="https://raw.githubusercontent.com/tamedia-ddj/brennpunkt_bauernhof_public/master/files/Bauernhof_Logo.jpg">

# Brennpunkt Bauernhof
## Analyse des Datenbestandes zu den Direktzahlungen 2014 bis 2017

Ein Rechercheteam von Tamedia veröffentlich diese Woche eine Serie von Artikeln zur Landwirtschaft. Es wurden dazu Daten über Direktzahlung sowie über 600 Strafurteile gegen Landwirte ausgewertet.

In den folgenden Notebooks sind die relevanten Ausschnitte der Analyse dokumentiert. Der beiliegende Datensatz wurde zur Wahrung des Datenschutzes gekürzt und verändert (Gemeinde, Kantone und Zahlenwerte wurden willkürlich vertauscht).




1. [Datenvorbereitung](https://github.com/tamedia-ddj/brennpunkt_bauernhof_public/blob/master/0_Data_Preparation.ipynb)
2. **Kürzungen**
    * Artikel:
        * [Skandalöse Zustände auf Schweizer Bauernhöfen](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/skandaloese-zustaende-auf-schweizer-bauernhoefen/story/17687029)
        * [Sogar Tierquäler erhalten Subventionen](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/bauern-erhalten-subventionen-trotz-leidender-tiere/story/19918846)
3. Kontrollen
	* Artikel:
		* [Zu wenig Kontrollen: Tierquäler bleiben unentdeckt](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/zu-wenig-geld-fuer-tierkontrollen/story/12827795)
        * [Der Bauern-Sherlock](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/wie-ein-ermittler-mit-videobeweisen-landwirte-ueberfuehrt/story/16595684)
4. Direktzahlungsbeträge
    * Methodik: [Github](https://github.com/tamedia-ddj/brennpunkt_bauernhof_public/blob/master/2_Betraege.ipynb)
    * Artikel: [«Das war für uns wie ein Lottosechser»](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/welche-bauern-am-meisten-erhalten/story/25753875)
5. Interview
	* Artikel: [«Beim Tierschutz gibt es keine Schmerzgrenze»](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/beim-tierschutz-gibt-es-keine-schmerzgrenze/story/20647149)
6. Interaktiv
    * Artikel: [Ein Leben lang trächtig und am Ende eine Wurst](https://interaktiv.tagesanzeiger.ch/2019/leben-einer-kuh/)
7. Tiertransporte
    * Artikel: [Die letzte Reise von Schwein Fanny](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/die-letzte-reise-von-schwein-fanny/story/14773958)
8. Verendungen
    * Artikel: [Zu krank für den Schlachthof](https://www.tagesanzeiger.ch/sonntagszeitung/zu-krank-fuer-den-schlachthof/story/28505215)
9. Leitartikel
    * Artikel: [Köpfe einschlagen bringt nichts](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/koepfe-einschlagen-bringt-nichts/story/26915768)


Quelle des Datensatzes: BLW

---

### Notebook 2 - Kürzungen
#### Berechnungen zu Artikeln

---

#### Load Packages

In [2]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

#### Import data

In [23]:
# read pickle-file
data = pd.read_pickle("files/data_mod.pkl")
# import fixed list of municipalities and their corresponding cantons
clean_gde = pd.read_pickle("files/gemeinden_bereinigt.pkl")

---

#### Calculate key facts

##### sum of all cutbacks in 2017

In [4]:
# sum of all cutbacks in 2017
print(data[data.JAHR == 2017].KUERZUNGEN.sum())

8114199.1


##### number of farms overall and with cutbacks in 2017

In [5]:
print("Alle Höfe: {}".format(len(data[data.JAHR == 2017].BETRIEBSNUMMER_ZUFALL.unique())))
print("Mit Kürzungen: {}".format(len(data[(data.JAHR == 2017) & (data.KUERZUNGEN > 0)].BETRIEBSNUMMER_ZUFALL.unique())))
print("Jeder {0:.0f}. Hof" .format(52192/7398))

Alle Höfe: 52192
Mit Kürzungen: 7398
Jeder 7. Hof


##### number of farms which received subsidies in the full period

In [6]:
print("Im Zeitraum 2014 - 2017 erhielten insgesamt {} Betriebe Gelder vom Bund.".format(len(data.BETRIEBSNUMMER_ZUFALL.unique())))

Im Zeitraum 2014 - 2017 erhielten insgesamt 62003 Betriebe Gelder vom Bund.


##### number of farms with cutbacks in the full period


In [13]:
print("{} Betrieben wurden Direktzahlungen gekürzt.".format(len(data[data.KUERZUNGEN > 0].BETRIEBSNUMMER_ZUFALL.unique())))

18313 Betrieben wurden Direktzahlungen gekürzt.


##### farms with multiple cutbacks

In [14]:
anzahl_kuerzungen = pd.DataFrame(data[(data.KUERZUNGEN > 0)].BETRIEBSNUMMER_ZUFALL.value_counts()).reset_index()
anzahl_kuerzungen.columns = ["BETRIEBSNUMMER_ZUFALL", "Anzahl_Kuerzungen"]

In [43]:
### >1
print("Anzahl Betriebe mit mehreren Kürzungen: {}".format(len(anzahl_kuerzungen[anzahl_kuerzungen.Anzahl_Kuerzungen > 1])))

Anzahl Betriebe mit mehreren Kürzungen: 4614


In [20]:
### count by number of cutbacks
anzahl_kuerzungen.Anzahl_Kuerzungen.value_counts().rename_axis('n_cutbacks').reset_index(name='counts')

,n_cutbacks,counts
0,1,13699
1,2,3550
2,3,869
3,4,195


---

#### cutbacks by municipality

In [24]:
### get number of farm with cutbacks by municipality
cutbacks_gde = data[(data.KUERZUNGEN > 0)][["GDE_GEMEINDE_NR", "KUERZUNGEN"]].groupby(["GDE_GEMEINDE_NR"]).count().reset_index()


In [26]:
### collect matching name and canton
cutbacks_gde = pd.merge(clean_gde, cutbacks_gde, on="GDE_GEMEINDE_NR", how="left")

In [27]:
### replace NAs
cutbacks_gde.KUERZUNGEN.fillna(0, inplace=True)

In [28]:
### function to get total number of farms by GDE_GEMEINDE_NR
def get_total_betriebe(gde_nr):
    return len(data[data.GDE_GEMEINDE_NR == gde_nr].BETRIEBSNUMMER_ZUFALL.unique())

In [29]:
### apply get_total_betriebe()
cutbacks_gde["BETRIEBE_TOTAL"] = cutbacks_gde.GDE_GEMEINDE_NR.apply(get_total_betriebe)

In [30]:
### calculate percentage of farms with cutbacks
cutbacks_gde["KUERZUNGEN_ANTEIL"] = cutbacks_gde.KUERZUNGEN / cutbacks_gde.BETRIEBE_TOTAL

In [31]:
### function to get average cutback by GDE_GEMEINDE_NR
def get_avg_kuerzung(gde_nr):
    return data[data.GDE_GEMEINDE_NR == gde_nr].KUERZUNGEN.mean()

In [32]:
### apply get_avg_kuerzung()
cutbacks_gde["KUERZUNGEN_BETRAG_AVG"] = cutbacks_gde.GDE_GEMEINDE_NR.apply(get_avg_kuerzung)

In [33]:
### function to clean municipality name / remove canton in parentheses
import re
def remove_kte(gde):
    return re.sub(r" ?\([^)]+\)", "", gde)

In [35]:
### apply remove_kte()
cutbacks_gde["GDE_CLEAN"] = cutbacks_gde.GDE_NAME.apply(remove_kte)

In [41]:
### look at sample row
cutbacks_gde[cutbacks_gde.index == 452]

,GDE_GEMEINDE_NR,GDE_NAME,ABJ_KANTON,KUERZUNGEN,BETRIEBE_TOTAL,KUERZUNGEN_ANTEIL,KUERZUNGEN_BETRAG_AVG,GDE_CLEAN
452,1010,Escholzmatt-Marbach,LU,83.0,330,0.251515,41.405611,Escholzmatt-Marbach


In [36]:
### Export relevant data to csv for visualization
cutbacks_gde[["KUERZUNGEN", "GDE_NAME", "BETRIEBE_TOTAL", "KUERZUNGEN_BETRAG_AVG", "ABJ_KANTON", "GDE_CLEAN"]].to_csv("data_output/n2_final.csv")


In [56]:
### Look at export
cutbacks_gde[["KUERZUNGEN", "GDE_NAME", "BETRIEBE_TOTAL", "KUERZUNGEN_BETRAG_AVG", "ABJ_KANTON", "GDE_CLEAN"]].head(10)

,KUERZUNGEN,GDE_NAME,BETRIEBE_TOTAL,KUERZUNGEN_BETRAG_AVG,ABJ_KANTON,GDE_CLEAN
0,7.0,Aeugst am Albis,14,303.235577,ZH,Aeugst am Albis
1,16.0,Affoltern am Albis,23,361.000633,ZH,Affoltern am Albis
2,10.0,Bonstetten,14,270.619048,ZH,Bonstetten
3,29.0,Hausen am Albis,45,320.239362,ZH,Hausen am Albis
4,6.0,Hedingen,13,109.026829,ZH,Hedingen
5,14.0,Kappel am Albis,24,255.989873,ZH,Kappel am Albis
6,15.0,Knonau,23,218.288095,ZH,Knonau
7,6.0,Maschwanden,20,240.514706,ZH,Maschwanden
8,17.0,Mettmenstetten,44,187.509236,ZH,Mettmenstetten
9,11.0,Obfelden,18,181.833077,ZH,Obfelden


---

#### localize farms with multiple cutbacks and prepare data for visualization

In [44]:
### mark farms with more than 2 cutbacks
anzahl_kuerzungen["multiple_cutbacks"] = 0
anzahl_kuerzungen.loc[anzahl_kuerzungen.Anzahl_Kuerzungen == 3, "multiple_cutbacks"] = 1  
anzahl_kuerzungen.loc[anzahl_kuerzungen.Anzahl_Kuerzungen == 4, "multiple_cutbacks"] = 1 

In [45]:
### assign municipality
cutbacks = pd.merge(data[["GDE_GEMEINDE_NR", "BETRIEBSNUMMER_ZUFALL"]].drop_duplicates(), anzahl_kuerzungen, how="left", on="BETRIEBSNUMMER_ZUFALL")

In [47]:
### replace NAs
cutbacks.loc[np.isnan(cutbacks.multiple_cutbacks), "multiple_cutbacks"] = 0 

In [48]:
### collect name and canton
cutbacks = pd.merge(clean_gde, cutbacks, on="GDE_GEMEINDE_NR", how="left")

In [49]:
### apply remove_kte()
cutbacks["GDE_CLEAN"] = cutbacks.GDE_NAME.apply(remove_kte)

In [50]:
### subset data
cutbacks = cutbacks[cutbacks.multiple_cutbacks != 0]

In [51]:
### save number of farms with multiple cutbacks by municipality
cutbacks[["GDE_GEMEINDE_NR", "GDE_NAME", "GDE_CLEAN", "ABJ_KANTON", "multiple_cutbacks"]].groupby(["GDE_GEMEINDE_NR", "GDE_NAME", "GDE_CLEAN", "ABJ_KANTON"]).sum().reset_index().to_csv("data_output/n3_final.csv")


In [57]:
### look at export
cutbacks[["GDE_GEMEINDE_NR", "GDE_NAME", "GDE_CLEAN", "ABJ_KANTON", "multiple_cutbacks"]].groupby(["GDE_GEMEINDE_NR", "GDE_NAME", "GDE_CLEAN", "ABJ_KANTON"]).sum().reset_index().tail(10)

,GDE_GEMEINDE_NR,GDE_NAME,GDE_CLEAN,ABJ_KANTON,multiple_cutbacks
558,6298,Visperterminen,Visperterminen,VS,5.0
559,6421,La Chaux-de-Fonds,La Chaux-de-Fonds,NE,1.0
560,6422,Les Planchettes,Les Planchettes,NE,1.0
561,6423,La Sagne,La Sagne,NE,1.0
562,6431,Les Brenets,Les Brenets,NE,1.0
563,6512,Val-de-Travers,Val-de-Travers,NE,3.0
564,6607,Bernex,Bernex,GE,3.0
565,6639,Soral,Soral,GE,1.0
566,6712,Develier,Develier,JU,1.0
567,6778,Bure,Bure,JU,1.0
